# Potential natural regeneration areas validation

This jupyter-notebook is parto of the project and is used to validate the process of defining the potential natural regeneration areas:

>This script is rganized to produce/identify areas for potential natural regeneration:
Grass and agricultural lands (omitting natural grass lands on grasslands layer), which are not supposed to be restorable as they area [natural] grass lands.

>Overlap this natural grasslands with ESA CCI 2000
~~agriculture and~~ pasture [,join with agriculture class] and then
get covariables for the random points.

**Latter by whatsapp:**
> Using NatGrass remove pixels that overlaps ESA2000 grasslands :. keeping only not natural grass lands which in deed was supposed to be restored.
Then, 'merge' with agriculture;

In [1]:
import ee
from ee import batch
from pprint import pprint
import webbrowser
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [2]:
# init EE
ee.Initialize()

# Importing data
NaturalGrass = ee.Image("users/HotSpotRestoration/PotentialNaturalRegeneration/NatGrass1992")
CCI = ee.Image("users/HotSpotRestoration/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992_2015-v207")
elevation = ee.Image("USGS/SRTMGL1_003")
world = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

In [14]:
# masking where there is Natural Grass pixels
NaturalGrassUnMask = NaturalGrass.unmask()
NaturalGrassInverted = NaturalGrassUnMask.eq(0)

# Organizing data
CCI2000 = CCI.select(['b9'], ['PotRestArea00']) #Select landuse for year 2000
# Masking
CCI2000 = CCI2000.updateMask(elevation)

pasto = CCI2000.eq(130)
pasto = pasto.mask(pasto)
PotArea2 = CCI2000.eq(130).And(NaturalGrassInverted)

# Create a folium map object.
myMap = folium.Map(location=[-29.30, -51], zoom_start=7)

# Add the elevation model to the map object.
myMap.add_ee_layer(pasto, {'min':0, 'max':1, 'palette':['white','blue']}, 'ESA CII grassland')
myMap.add_ee_layer(NaturalGrass.mask(NaturalGrass), {'min':0, 'max':1, 'palette':['red','black']}, 'Natural Grasslands - IIS')
myMap.add_ee_layer(PotArea2.mask(PotArea2), {'min':0, 'max':1, 'palette':['red','red']}, 'not natural Grassland')
myMap.add_child(folium.LayerControl())

myMap